# Lab 04 · Structured JSON and Validation

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A planner endpoint is produced that returns structured JSON.
- Pydantic validation is applied to enforce schema guarantees.
- Automatic repair strategies are outlined for invalid JSON.

## What will be learned
- Structured JSON responses are validated on the backend.
- Error handling flows for invalid planner output are reviewed.
- Lightweight repair attempts are documented for client consumption.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install pydantic
```

## Step-by-step tasks
### Step 1: Planner schema definition
A schema is defined so planner responses remain consistent.

In [ ]:
from pathlib import Path
planner_path = Path("ai-web/backend/app/planner.py")
planner_path.write_text('''from pydantic import BaseModel, Field, ValidationError
from typing import List


class Plan(BaseModel):
    goal: str = Field(..., description="High level objective")
    steps: List[str] = Field(default_factory=list, description="Ordered plan steps")


def build_plan(goal: str) -> Plan:
    steps = [
        "It is ensured that the goal is clarified.",
        "Resources are gathered to support the plan.",
        "Progress is reviewed upon completion.",
    ]
    return Plan(goal=goal, steps=steps)


def repair_plan(data: dict) -> Plan:
    try:
        return Plan(**data)
    except ValidationError as exc:
        fixed = {"goal": data.get("goal", "A goal was recorded."), "steps": []}
        for idx, issue in enumerate(exc.errors()):
            fixed["steps"].append(f"Step {idx + 1} was replaced because {issue['msg']}")
        return Plan(**fixed)
''')
print("Planner module was written.")

### Step 2: API exposure
The planner endpoint is exposed at /api/plan with validation.

In [ ]:
from pathlib import Path
main_path = Path("ai-web/backend/app/main.py")
text = main_path.read_text()
if "plan_endpoint" not in text:
    addition = '''
from pydantic import BaseModel
from .planner import build_plan, repair_plan


class PlanIn(BaseModel):
    goal: str


@app.post("/api/plan")
def plan_endpoint(payload: PlanIn):
    plan = build_plan(payload.goal)
    return plan.model_dump()


@app.post("/api/plan/repair")
def plan_repair_endpoint(payload: dict):
    plan = repair_plan(payload)
    return plan.model_dump()
'''
    main_path.write_text(text.rstrip() + "
" + addition)
    print("Planner routes were appended.")
else:
    print("Planner routes already present.")

## Validation / acceptance checks
```bash
# locally
curl -X POST http://localhost:8000/api/plan -H 'Content-Type: application/json' -d '{"goal":"Build a demo"}'
```
- A JSON object containing a goal and an ordered list of steps is returned.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- Client-side rendering of planner steps is drafted for the frontend.
- Additional validation rules are explored for complex goals.